In [1]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from block2 import SU2 as block2_SU2
from block2 import SZ as block2_SZ
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from DMRG_simulation.dmrg_simulation import get_ground_state_fci
import numpy as np
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.var_utils as varu
import CAS_Cropping.matrix_utils as matru
import pyscf
import scipy
import openfermion as of

In [36]:
def custom_obt():
    """
    
    Returns:

    """
    tensor = np.zeros((2, 2))
    for p in range(2):
        for q in range(2):
            tensor[p ,q] = 1
    
    #alternative = np.array([[-1.8351087041237648, 0.3, 0, 0], [0.3, -1.5506525118237189, 6.71193502e-02, 0], [0, 6.71193502e-02, -1.2458017459416912, 2.86937432e-03], [0, 0,
    # #2.86937432e-03, -0.9063250085305642]])
    alternative = np.array([[0, 0.3, 6.71193502e-02, 0.2], [0.3, 0, 6.71193502e-02, 0.3], [6.71193502e-02, 6.71193502e-02, 0, 2.86937432e-03], [0.2, 0.3, 2.86937432e-03, 0]])
    #alternative = np.array([[-1.8351087041237648, 0, 0, 0, 0], [0, -1.5506525118237189, 0, 0, 0], 
    #                         [0, 0, -1.2458017459416912, 0, 0], [0, 0, 0, -0.9063250085305642, 0], [0, 0, 0, 0, -0.9063250085305642]])
    #alternative = np.array([[-1.8351087041237648, 0, 0, 0, 0, 0, 0], [0, -1.5506525118237189, 0, 0, 0, 0, 0], 
    #                        [0, 0, -1.2458017459416912, 0, 0, 0, 0], [0, 0, 0, -0.9063250085305642, 0, 0, 0], [0, 0, 0, 0, -0.9063250085305642, 0, 0], 
    #                        [0, 0, 0, 0, 0, -1.2458017459416912, 0], [0, 0, 0, 0, 0, 0, -1.2458017459416912],])
    #alternative = np.array([[-1.11084418 , 0], [0, -0.589121]])
    #alternative = np.array(([-1.541249701637053 , 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]))
    #alternative = [np.array([[-1.541249701637053, 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]]),
    #              np.array([[-1.541249701637053, 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]])]
    # alternative = [
    #     np.array([[-1.11084418 , 0], [0, -0.589121]]),
    #     np.array([[-1.11084418 , 0], [0, -0.589121]])
    # ]
    return alternative

def orbtransf(tensor, U, complex = False):
    """Return applying UHU* for the tensor representing the 1e or 2e tensor"""
    if len(tensor.shape) == 4:
        p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor)[0]
        return np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor, optimize = p)
    elif len(tensor.shape) == 2:
        p = np.einsum_path('ap,bq, pq->ab', U, U, tensor)[0]
        return np.einsum('ap,bq, pq->ab', U, U, tensor, optimize = p)

In [37]:
n = 4
random_uparams = np.random.rand(n*n)
U = matru.construct_orthogonal(n, random_uparams)
zero_tensor = np.zeros((n, n, n, n))
orb_trans = orbtransf(custom_obt(), U, complex = True)
one_body_tensor =  [custom_obt(), custom_obt()]
two_body_tensor = [zero_tensor, zero_tensor, zero_tensor]

print(one_body_tensor[0])

[[0.         0.3        0.06711935 0.2       ]
 [0.3        0.         0.06711935 0.3       ]
 [0.06711935 0.06711935 0.         0.00286937]
 [0.2        0.3        0.00286937 0.        ]]


In [38]:
from DMRG_simulation.format_dmrg.format_dmrg_param import get_dmrg_param, get_dmrg_process_param


default_final_bond_dim = 1
default_sweep_schedule_bond_dims = [default_final_bond_dim] * 4 + [
    default_final_bond_dim+31
] * 4
default_sweep_schedule_noise = [1e-10] * 4 + [1e-10] * 4 + [1e-15]
default_sweep_schedule_davidson_threshold = [1e-20] * 8

init_state_bond_dimension = 10
max_num_sweeps = 200
energy_convergence_threshold = 1e-8
sweep_schedule_bond_dims = default_sweep_schedule_bond_dims
sweep_schedule_noise = default_sweep_schedule_noise
sweep_schedule_davidson_threshold = (
    default_sweep_schedule_davidson_threshold
)
nuc_rep_energy = 0
    
num_orbitals = 4
num_electrons = 2
num_unpaired = 0
multiplicity = 1

dmrg_process_param = get_dmrg_process_param(
    init_state_bond_dimension, max_num_sweeps, energy_convergence_threshold,
    sweep_schedule_bond_dims, sweep_schedule_noise,
    sweep_schedule_davidson_threshold)
dmrg_param = get_dmrg_param(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity, dmrg_process_param)

result = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(one_body_tensor, two_body_tensor, dmrg_param)
mps = result['ket_optimized']
print(result)

dtrie finished 0.0022905829991941573
Number of DET =         16 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 a0b0  =    0.526900632087058
DET          1 a00b  =    0.505830597498774
DET          2 2000  =    0.473726225197478
DET          3 ab00  =   -0.234542928631088
DET          4 0a0b  =    0.210464487436181
DET          5 00ba  =    0.192589712969422
DET          6 0ab0  =   -0.184479743319398
DET          7 0020  =    0.139368231302377
DET          8 0b0a  =    0.127340561382293
DET          9 ba00  =   -0.109184542700969
DET         10 0002  =   -0.102527463611823
DET         11 0200  =   -0.083200825765113
DET         12 00ab  =    0.064122471559506
DET         13 b00a  =    0.046688087277949
DET         14 b0a0  =    0.017561439200666
DET         15 0ba0  =    0.008415282800403
{'dmrg_driver': <pyblock2.driver.core.DMRGDriver object at 0x102e079a0>, 'dmrg_ground_state_energy': -0.6748264466071734, 'initial_ket': <block2.sz.MPS obje

In [39]:
loop_results = dmrg_central_loop(one_body_tensor, two_body_tensor, dmrg_param, 100, 1e6, 1e-15   , "./mps_directory", verbosity=2, move_mps_to_final_storage_path=None)


integral cutoff error =  0.0
mpo terms =         24

Build MPO | Nsites =     4 | Nterms =         24 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     5 DW = 0.00e+00 NNZ =        5 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     4 .. Mmpo =    10 DW = 0.00e+00 NNZ =       13 SPT = 0.7400 Tmvc = 0.000 T = 0.002
 Site =     2 /     4 .. Mmpo =     5 DW = 0.00e+00 NNZ =       21 SPT = 0.5800 Tmvc = 0.000 T = 0.001
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        5 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.006 Tmvc-total = 0.000 MPO bond dimension =    10 MaxDW = 0.00e+00
NNZ =           44 SIZE =          110 SPT = 0.6000

Rank =     0 Ttotal =      0.015 MPO method = FastBipartite bond dimension =      10 NNZ =           44 SIZE =          110 SPT = 0.6000
dtrie finished 0.0011840000006486662
Number of DET =         16 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 a0b0  =    0.52

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


In [43]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")

In [44]:
mps_dir = Path("./mps_directory/mps_storage/dmrg_loop_001_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)

In [45]:
tools.init(block2_SZ)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)

cp: mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_001_ket_optimized/F.MPS.INFO.ket_

In [46]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)


In [47]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)

In [48]:
ener = driver.expectation(mps, mpo, mps)
print(ener)

bra = driver.get_random_mps(tag="GS5", bond_dim=250, nroots=1)
applied_mps = driver.multiply(bra ,mpo, mps)
print(applied_mps)

variance = np.sqrt(abs(applied_mps ** 2 - ener ** 2))
print(f"Variance: {variance}")
print(f"Variance ratio: {variance / ener}")

-6.7715224318949705
6.771522431894969
Variance: 1.4600096599955427e-07
Variance ratio: -2.156102522999348e-08


In [49]:
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)


mps center changed (temporarily)
dtrie finished 0.0020044999999981883
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000001

DET          0 2200  =   -0.496983597366318
DET          1 a2b0  =    0.370240831637947
DET          2 b2a0  =   -0.370240831634914
DET          3 0220  =   -0.275820518299222
DET          4 2a0b  =    0.247270798132030
DET          5 2b0a  =   -0.247270798127969
DET          6 abba  =   -0.184210799764489
DET          7 baab  =   -0.184210799762641
DET          8 aabb  =    0.158001119372249
DET          9 bbaa  =    0.158001119372064
DET         10 2ab0  =    0.128681532185570
DET         11 2ba0  =   -0.128681532184069
DET         12 2002  =   -0.123027898567950
DET         13 0a2b  =    0.117707035304423
DET         14 0b2a  =   -0.117707035294581
DET         15 b20a  =   -0.101224946762029
DET         16 a20b  =    0.101224946760845
DET         17 ab20  =    0.095864647739749
DET         18 ba20  =   -0.0958646